# Technical Analysis Trading Strategy

This notebook implements a trading strategy using four technical indicators:
1. RSI (Relative Strength Index)
2. Range Detector
3. McGinley Dynamic
4. Damiani Volatility

The strategy combines these indicators to generate trading signals with proper risk management.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import talib as ta
from lightweight_charts import Chart
import datetime
import asyncio
import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings('ignore')

# For plotting
plt.style.use('classic')  # Using a built-in style
plt.rcParams['figure.figsize'] = (15, 8)

In [2]:
# Load stock data from yfinance
def fetch_stock_data(symbol, period="2y", interval="1d"):
    """
    Fetch stock data from yfinance with error handling
    
    Parameters:
    symbol (str): Stock symbol (e.g., 'AAPL', 'MSFT', 'GOOGL', 'SPY')
    period (str): Data period ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max')
    interval (str): Data interval ('1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo')
    """
    try:
        print(f"📈 Fetching {period} of {interval} data for {symbol}...")
        ticker = yf.Ticker(symbol)
        df = ticker.history(period=period, interval=interval)
        
        if df.empty:
            raise ValueError(f"No data found for symbol {symbol}")
        
        # Basic preprocessing
        df = df[df['Volume'] != 0]  # Remove zero volume days
        df.columns = [col.lower() for col in df.columns]  # Lowercase columns
        
        print(f"✅ Successfully loaded {len(df)} data points")
        print(f"📅 Date range: {df.index[0].date()} to {df.index[-1].date()}")
        print(f"💰 Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error fetching data: {e}")
        print("💡 Try different symbols like: AAPL, MSFT, GOOGL, TSLA, NVDA, SPY, QQQ")
        return None



In [3]:
# Configuration - Change these parameters as needed
SYMBOL = 'SPY'      # Stock symbol to analyze
PERIOD = '2y'        # Data period (1y, 2y, 5y, etc.)
INTERVAL = '1d'      # Data interval (1d for daily, 1h for hourly, etc.)

# Fetch the data
df = fetch_stock_data(SYMBOL, period=PERIOD, interval=INTERVAL)

if df is not None:
    print(f"\n📊 Data Summary for {SYMBOL}:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"\nFirst 5 rows:")
    print(df.head())
    print(f"\nLast 5 rows:")
    print(df.tail())
    
    # Check for missing values
    print(f"\nMissing values:")
    print(df.isna().sum())
else:
    print("❌ Failed to load data. Please check your internet connection and symbol.")

# Save stock data for future use
df.to_csv("{SYMBOL}.csv", index=True)



📈 Fetching 2y of 1d data for SPY...
✅ Successfully loaded 502 data points
📅 Date range: 2023-10-18 to 2025-10-17
💰 Price range: $400.33 - $673.11

📊 Data Summary for SPY:
Shape: (502, 8)
Columns: ['open', 'high', 'low', 'close', 'volume', 'dividends', 'stock splits', 'capital gains']

First 5 rows:
                                 open        high         low       close  \
Date                                                                        
2023-10-18 00:00:00-04:00  423.251323  424.216373  418.279803  419.371582   
2023-10-19 00:00:00-04:00  420.092912  421.915796  415.004421  415.686768   
2023-10-20 00:00:00-04:00  415.248128  415.794018  410.471552  410.578796   
2023-10-23 00:00:00-04:00  409.038626  413.756717  407.274229  409.867218   
2023-10-24 00:00:00-04:00  412.002015  414.117359  410.140131  412.957336   

                              volume  dividends  stock splits  capital gains  
Date                                                                          
20

In [4]:
# Load stock data
symbol = 'SPY'  # Replace with your actual symbol
df = pd.read_csv(f"{symbol}.csv", parse_dates=['Date'])  # Parse dates during CSV reading
df = df.drop(columns=['dividends', 'stock splits', 'capital gains'], errors='ignore')
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279190,85.324337,83.909667,84.556805,118944600
1,2010-01-05,85.504951,85.542578,84.918014,85.226535,111579900
2,2010-01-06,85.565140,85.775835,85.354445,85.422166,116074400
3,2010-01-07,85.926353,86.031701,85.166341,85.407136,131091100
4,2010-01-08,86.212280,86.249907,85.527521,85.700590,126402800


In [5]:
# Define Custom Indicators
def mcginley_dynamic(close_prices, n=14):
    md = pd.Series(index=close_prices.index, dtype='float64')
    md.iloc[0] = close_prices.iloc[0]
    for i in range(1, len(close_prices)):
        if md.iloc[i-1] != 0 and not pd.isna(md.iloc[i-1]):
            denominator = (n * (close_prices.iloc[i] / md.iloc[i-1]) ** 4)
            if denominator != 0:
                md.iloc[i] = md.iloc[i-1] + (close_prices.iloc[i] - md.iloc[i-1]) / denominator
            else:
                md.iloc[i] = md.iloc[i-1]
        else:
            md.iloc[i] = close_prices.iloc[i]
    return md

def damiani_volatmeter(high, low, close, viscosity=7, sedimentation=100, threshold_level=1.4, matype='sma'):
    typical_price = (high + low + close) / 3
    atr_viscosity = ta.ATR(high, low, close, timeperiod=viscosity)
    atr_sedimentation = ta.ATR(high, low, close, timeperiod=sedimentation)
    vol = atr_viscosity / atr_sedimentation
    if matype.lower() == 'ema':
        ma_viscosity = typical_price.ewm(span=viscosity, adjust=False).mean()
        ma_sedimentation = typical_price.ewm(span=sedimentation, adjust=False).mean()
    elif matype.lower() == 'wma':
        ma_viscosity = ta.wma(typical_price, length=viscosity)
        ma_sedimentation = ta.wma(typical_price, length=sedimentation)
    else: 
        ma_viscosity = typical_price.rolling(window=viscosity).mean()
        ma_sedimentation = typical_price.rolling(window=sedimentation).mean()
    anti = typical_price.rolling(window=viscosity).apply(lambda x: (x - ma_viscosity.loc[x.index]).std(), raw=False)
    sedi = typical_price.rolling(window=sedimentation).apply(lambda x: (x - ma_sedimentation.loc[x.index]).std(), raw=False)
    antithres = anti / sedi
    threshold = threshold_level - antithres
    result_df = pd.DataFrame({
        'volatility_meter': vol,
        'threshold_level': threshold
    }, index=close.index)
    return result_df



In [6]:
# Calculate RSI
df['rsi'] = ta.RSI(df['Close'], timeperiod=14)
df['rsi_ma12'] = df['rsi'].rolling(12).mean()
df['bullishRSI'] = 0.0
df['bullishRSI'] = np.where(df['rsi'] > df['rsi_ma12'], 1.0, 0.0)
df['crossover_RSI'] = df['bullishRSI'].diff()

# Calculate McGinley Dynamic
df['mcginley'] = mcginley_dynamic(df['Close'], n=13)
df['bullishMcGinley'] = 0.0
df['bullishMcGinley'] = np.where(df['Close'] > df['mcginley'], 1.0, 0.0)
df['crossover_McGinley'] = df['bullishMcGinley'].diff() 


# Calculate Damiani Volatmeter
damiani_df = damiani_volatmeter(df['High'], df['Low'], df['Close'])


df = pd.concat([df, damiani_df], axis=1)
df = df.dropna().reset_index(drop=True)
df.tail()


,Date,Close,High,Low,Open,Volume,rsi,rsi_ma12,bullishRSI,crossover_RSI,mcginley,bullishMcGinley,crossover_McGinley,volatility_meter,threshold_level
3867,2025-10-10,653.020020,673.950012,652.840027,672.130005,159422600,42.498581,65.006459,0.0,0.0,659.872906,0.0,-1.0,1.027398,1.002590
3868,2025-10-13,663.039978,665.130005,659.770020,660.650024,79560500,52.632831,64.458754,0.0,0.0,660.111906,1.0,1.0,1.125919,0.967666
3869,2025-10-14,662.229980,665.830017,653.169983,657.169983,88779600,51.837479,63.524600,0.0,0.0,660.272760,1.0,0.0,1.218326,0.960906
3870,2025-10-15,665.169983,670.229980,658.929993,666.820007,81702600,54.523669,62.667550,0.0,0.0,660.638498,1.0,0.0,1.269508,1.003678
3871,2025-10-16,660.640015,668.710022,657.109985,666.820007,110409600,49.905127,61.236384,0.0,0.0,660.638614,1.0,0.0,1.317331,1.020125


In [7]:
matching_rows = df[df['crossover_RSI'] == df['crossover_McGinley']]
matching_rows

,Date,Close,High,Low,Open,Volume,rsi,rsi_ma12,bullishRSI,crossover_RSI,mcginley,bullishMcGinley,crossover_McGinley,volatility_meter,threshold_level
1,2010-05-28,82.638283,83.658321,82.245376,83.597873,297933500,40.504239,37.691386,1.0,0.0,84.775143,0.0,0.0,1.647387,-0.699902
2,2010-06-01,81.248001,83.076514,81.127111,81.867580,277909400,37.099458,36.936946,1.0,0.0,84.453552,0.0,0.0,1.609807,-0.649152
3,2010-06-02,83.363640,83.371192,81.232892,81.663575,240243700,44.715286,37.197494,1.0,0.0,84.365242,0.0,0.0,1.594427,-0.564704
4,2010-06-03,83.650757,83.915210,82.796948,83.605424,226618300,45.676570,37.523383,1.0,0.0,84.308379,0.0,0.0,1.485696,-0.261828
5,2010-06-04,80.711533,82.608051,80.439522,82.064029,398475600,38.329154,37.507861,1.0,0.0,83.978983,0.0,0.0,1.586701,0.273648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3862,2025-10-03,669.210022,672.679993,668.159973,669.989990,70494400,69.669654,67.668768,1.0,0.0,656.768502,1.0,0.0,0.715883,1.146458
3863,2025-10-06,671.609985,672.510010,669.460022,671.619995,54623300,71.829986,67.821769,1.0,0.0,657.812536,1.0,0.0,0.686873,1.219762
3869,2025-10-14,662.229980,665.830017,653.169983,657.169983,88779600,51.837479,63.524600,0.0,0.0,660.272760,1.0,0.0,1.218326,0.960906
3870,2025-10-15,665.169983,670.229980,658.929993,666.820007,81702600,54.523669,62.667550,0.0,0.0,660.638498,1.0,0.0,1.269508,1.003678


In [8]:
# Relative Strength Index (RSI)
if __name__ == '__main__':
    
    chart = Chart(title="Relative Strength Index (RSI)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # Create a copy of the dataframe with the index as a column for the chart
    chart_df = df.reset_index()
    chart.set(chart_df)

    # Create line series for RSI
    rsi_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    rsi_line = rsi_chart.create_line('rsi', color='#ffeb3b', width=1, price_line=False, price_label=False)
    rsi_line.set(chart_df[['Date', 'rsi']])

    rsi_signal_line = rsi_chart.create_line('rsi_ma12', color='#26c6da', width=1, price_line=False, price_label=False)
    rsi_signal_line.set(chart_df[['Date', 'rsi_ma12']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(chart_df)):
        rsi_diff = chart_df.iloc[i]['crossover_RSI']
        current_time = chart_df.iloc[i]['Date']

        # Check for buy signal (RSI crosses above MA)
        if rsi_diff == 1:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (RSI crosses below MA)
        elif rsi_diff == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

    # Show the chart without blocking
chart.show(block=True)

In [9]:
if __name__ == '__main__':

    chart = Chart(title="Mcginley Dynamic 20", maximize=True)
    chart.legend(visible=True)
    chart.set(df)

    mcginley_line = chart.create_line('mcginley', color='#26c6da', width=1, price_label=True)
    mcginley_line.set(df[['Date', 'mcginley']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        mc_diff = df.iloc[i]['crossover_McGinley']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (mcginley crosses above)
        if mc_diff == 1:
             markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })

        # Check for sell signal (mcginley crosses below)
        elif mc_diff == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
    chart.show(block = True)

In [15]:
if __name__ == '__main__':

    chart = Chart(title="RSI and McGinley Dynamic 20 with Stop Loss", maximize=True)
    chart.legend(visible=True)
    chart.set(df)

    mcginley_line = chart.create_line('mcginley', color='#26c6da', width=1, price_label=True)
    mcginley_line.set(df[['Date', 'mcginley']])

    # Initialize variables
    markers = []
    buy_signal = 0
    sell_signal = 0
    entry_price = 0
    stop_loss_pct = 0.10  # 10% stop loss

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        mc_diff = df.iloc[i]['crossover_McGinley']
        rsi_diff = df.iloc[i]['crossover_RSI']
        current_time = df.iloc[i]['Date']
        current_price = df.iloc[i]['Close']

        # Check for stop loss if we're in a long position
        if buy_signal == 1:
            stop_loss_price = entry_price * (1 - stop_loss_pct)
            if current_price <= stop_loss_price:
                markers.append({
                    'time': current_time,
                    'position': 'above',
                    'shape': 'arrow_down',
                    'color': '#ff0000',  # Red color for stop loss
                    'text': 'Long Stop Loss and SELL'
                })
                sell_signal = 1
                buy_signal = 0
                continue
        
        # Check for stop loss if we're in a short position
        if sell_signal == 1:
            stop_loss_price = entry_price * (1 + stop_loss_pct)
            if current_price >= stop_loss_price:
                markers.append({
                    'time': current_time,
                    'position': 'below',
                    'shape': 'arrow_up',
                    'color': '#ff0000',  # Red color for stop loss
                    'text': 'Short Stop Loss and BUY'
                })
                sell_signal = 0
                buy_signal = 1
                continue

        # Check for buy signal (mcginley crosses above close and rsi crosses above ma)
        if mc_diff == 1 and rsi_diff == 1 and buy_signal == 0:
             markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
             
             buy_signal = 1
             sell_signal = 0
             entry_price = current_price  # Record entry price for stop loss
             
        # Check for sell signal (mcginley crosses below close and rsi crosses below ma)
        elif mc_diff == -1 and rsi_diff == -1 and sell_signal == 0:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })
            sell_signal = 1
            buy_signal = 0
            entry_price = current_price  # Record entry price for stop loss

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
    chart.show(block = True)

In [11]:
# # 6. Backtesting
# df['returns'] = df['Close'].pct_change()
# df['strategy_returns'] = df['returns'] * df['signal'].shift(1)
# cumulative_returns = (1 + df['strategy_returns']).cumprod()

# print("Displaying Cumulative Strategy Returns Chart")
# plt.figure(figsize=(12, 6))
# plt.plot(cumulative_returns, label='Strategy Returns')
# plt.title('Cumulative Strategy Returns')
# plt.legend()
# plt.show()



In [12]:
# # 7. Visualize Trades
# print("Displaying Trading Strategy Signals Chart")
# plt.figure(figsize=(16, 8))
# plt.plot(df['close'], label='Close Price')
# plt.plot(df[df['signal'] == 1].index, df['close'][df['signal'] == 1], '^', markersize=10, color='g', lw=0, label='Buy Signal')
# plt.plot(df[df['signal'] == -1].index, df['close'][df['signal'] == -1], 'v', markersize=10, color='r', lw=0, label='Sell Signal')
# plt.plot(df['mcginley'], label='McGinley Dynamic', linestyle='--')
# plt.title('Trading Strategy Signals')
# plt.legend()
# plt.show()

# print("\\n--- Strategy Data ---")
# print(df.tail())